# Important informations

In [ ]:
# Important informations
StatesFull = ['Alabama','Alaska','Arizona','Arkansas','California', 'Colorado','Connecticut',
          'Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa',
          'Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota',
          'Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey',
          'New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon',
          'Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah',
          'Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']
StatesFull=[S.lower() for S in StatesFull]
print(len(StatesFull))
StatesTL=['AL','AK','AZ','AR','CA','CO','CT','DE','FL','KY','LA','ME','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK',
         'OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY','KS','GA','HI','MD','MA','ID','IL','IN','MI',
          'IA','MN','MS','MO']
StatesTL=[S.lower() for S in StatesTL]
print(len(StatesTL))
States_dict={'Alabama' : 'AL','Alaska' : 'AK','Arizona' : 'AZ','Arkansas' : 'AR','California': 'CA','Colorado' : 'CO',
             'Connecticut' : 'CT','Delaware' : 'DE','Florida' : 'FL','Georgia' : 'GA','Hawaii' : 'HI','Idaho' : 'ID',
             'Illinois' : 'IL','Indiana' : 'IN','Iowa' : 'IA', 'Kansas' : 'KS','Kentucky' : 'KY','Louisiana' : 'LA','Maine':'ME',
             'Maryland' : 'MD','Massachusetts': 'MA' ,'Michigan' : 'MI','Minnesota' : 'MN', 'Mississippi' : 'MS','Missouri' : 'MO',
             'Montana' : 'MT','Nebraska' : 'NE','Nevada' : 'NV','New hampshire' : 'NH','New jersey' : 'NJ','New mexico' : 'NM',
             'New york' : 'NY', 'North carolina' : 'NC','North dakota' : 'ND', 'Ohio' : 'OH', 'Oklahoma' : 'OK','Oregon' : 'OR',
             'Pennsylvania' : 'PA', 'Rhode island' : 'RI','South carolina' : 'SC','South dakota' : 'SD','Tennessee' : 'TN',
             'Texas' : 'TX', 'Utah' : 'UT', 'Vermont' : 'VT', 'Virginia' : 'VA','Washington' : 'WA', 'West virginia' : 'WV',
             'Wisconsin' :'WI', 'Wyoming':'WY'}
print(len(States_dict))

In [ ]:
import tweepy
import csv
import json
import pandas as pd
from collections import Counter

In [ ]:
# create a dictionary to store your twitter credentials

twitter_cred = dict()

# Enter your own consumer_key, consumer_secret, access_key and access_secret
# Replacing the stars ("********")

twitter_cred['CONSUMER_KEY'] = ''
twitter_cred['CONSUMER_SECRET'] = ''
twitter_cred['ACCESS_KEY'] = ''
twitter_cred['ACCESS_SECRET'] = ''

# Save the information to a json so that it can be reused in code without exposing
# the secret info to public

with open('twitter_credentials.json', 'w') as secret_info:
    json.dump(twitter_cred, secret_info, indent=4, sort_keys=True)

In [ ]:
# load Twitter API credentials
with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_key = info['ACCESS_KEY']
    access_secret = info['ACCESS_SECRET']

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Functions

In [ ]:
def scroll_tweets(query,limit=None):
        """
        returns all the tweets within 7 days top according to the query received by this method
        returns the complete tweet
        :param query: should contain all the words and can include logic operators
        should also provide the period of time for the search
        ex: rock OR axe 
        (visit https://dev.twitter.com/rest/public/search to see how to create a query)
        :param lang: the language of the tweets
        :param limit: defines the maximum amount of tweets to fetch
        :return: tweets: a list of all tweets obtained after the request
        """
        tweets = []

        for tweet in tweepy.Cursor(api.search, q=query).items(limit):
            JSON=tweet._json
            JSON['query'] = query
            tweets.append(JSON)
            #time.(delay)

        return tweets 


In [ ]:
def extract_information(tweets):
    df = pd.DataFrame(columns=['Query','TweetId','UserId','ScreenName', 'Date', 'Location', 'Coordinates', 'Place', 'Text'])
    i=0
    for tweet in tweets:
        try:
            
            d = [{'Query': tweet['query'],'TweetId': tweet['id'],'UserId':tweet['user']['id'],'ScreenName':tweet['user']['screen_name'], 
                   'Date':tweet['created_at'], 'Location':tweet['user']['location'], 'Coordinates':tweet['coordinates'],
                   'Place':tweet['place'], 'Text':tweet['text']}]
            newrow= pd.DataFrame(d)

            df=df.append(newrow, ignore_index=True)
            i=i+1
        except:
            print(i, 'exception')
            i=i+1

   
    return(df)

In [ ]:
# SPlit the content of the location column using a co
# Check if the Location contains the two-letter state code 
USNames=['united states', 'usa', 'us']
def determine_USState(all_tweets):
    StatesTest=[]
    for i,L in enumerate(all_tweets['Location']):
        L=L.strip().lower()
        if ',' in L: ## Orlando, FL
            test=L.split(',')[1] 
            test=test.strip()
            if test in StatesTL:
                StatesTest.append(test.capitalize())
            elif test in StatesFull:
                StatesTest.append(States_dict[test.capitalize()])
            elif test in USNames: # For these cases Florida, USA
                firstelement=L.split(',')[0]
                if firstelement in StatesTL:
                    StatesTest.append(firstelement.capitalize())
                elif firstelement in StatesFull:
                    StatesTest.append(States_dict[firstelement.capitalize()])
                else:
                    StatesTest.append('probably not US')
                    next  
            else:
                StatesTest.append('probably not US')
                next           
        elif L in StatesFull:
            StatesTest.append(States_dict[L.capitalize()])
        else:
            StatesTest.append('probably not US')

    all_tweets['State']=StatesTest
    all_tweets['State']=all_tweets['State'].str.lower()
    return(all_tweets)

# Restaurants
## Healthy restaurant chains (15)

In [ ]:
# Let's get some data 1000 tweets naming one of those restaurants
test=['Panerabread']
healthyRestaurants=['Panerabread','jasonsdeli','aubonpain','Noodles and Company','Chipotle','AtlantaBread', 'EinsteinBros',
                   'LePainQuotidien', 'Justsalad', 'Mymarthas','krunch','chopt', 'sweetgreen', 'cava','olivegarden']
print('Number of healthy Restaurants tested:',len(healthyRestaurants))
healthy_tweets_list=[]
for Rest in healthyRestaurants:
    healthy_tweets=scroll_tweets(query=Rest, limit=2000)
    healthy_tweets_list.append(healthy_tweets)
    
hRest_t_l = [item for sublist in healthy_tweets_list for item in sublist]
len(hRest_t_l)

In [ ]:
# Let's locate those tweets
healthy_tweet_df=extract_information(hRest_t_l)
healthy_tweet_df['Cat']='healthy'
healthy_tweet_df=determine_USState(healthy_tweet_df)
healthy_tweet_df.to_csv('healthyRestaurants_tweet_df_NEW.csv')

## Unhealthy restaurant chains (19)

In [ ]:
unhealthyRestaurants=['kfc','tacobell','burgerking','cinnabon','chickfila','PandaExpress','dunkindonuts','pizzahut','WaffleHouse',
                     'cinnabon','AutieAnnes','cheesecake','arbys','wendys', 'FiveGuys','Shakeshack','WhiteCastle','DairyQueen',
                     'quiznos']
print('Number of unhealthy Restaurants tested:',len(unhealthyRestaurants))
unhealthy_tweets_list=[]
for Rest in unhealthyRestaurants:
    unhealthy_tweets=scroll_tweets(query=Rest, limit=2000)
    unhealthy_tweets_list.append(unhealthy_tweets)
uhRest_t_l = [item for sublist in unhealthy_tweets_list for item in sublist]
len(uhRest_t_l)

In [ ]:
len(uhRest_t_l)

In [ ]:
# Let's locate those tweets
unhealthy_tweet_df=extract_information(uhRest_t_l)
unhealthy_tweet_df['Cat']='unhealthy'
unhealthy_tweet_df=determine_USState(unhealthy_tweet_df)
unhealthy_tweet_df.to_csv('unhealthyResturants_tweet_df_NEW.csv')

# Activities
## Healthy activities (44)

In [ ]:
healthyActivities=['run','running','ran','walk','walking','walked','hike','hiking','hiked','surf','surfing','yoga','exercise',
                  'climb','climbed','soccer','tennis','volleyball','baseball','softball','swim','swimming','swam','dance','ballet',
                  'mountain bike','marathon','triathlon','boxing','kick boxing','gymnastics','ski','skiing','snow boarding', 'snowboard',
                   'kanu','kayak','row','rowing','sail','sailing','sailed', 'body building', 'spinning', 'cardio']
print('Number of healthy Activities tested:',len(healthyActivities))
healthy_acts_list=[]
for act in healthyActivities:
    healthyact_tweets=scroll_tweets(query=act, limit=2000)
    healthy_acts_list.append(healthyact_tweets)
hAct_t_l = [item for sublist in healthy_acts_list for item in sublist]
len(hAct_t_l)

In [ ]:
healthyact_tweet_df=extract_information(hAct_t_l)
healthyact_tweet_df['Cat']='healthy'
healthyact_tweet_df=determine_USState(healthyact_tweet_df)
healthyact_tweet_df.to_csv('healthyACT_tweet_df_NEW.csv')

## Unhealthy Activities (38)

In [ ]:
unhealthyActivities=['couch','sofa','nap','sleep','TV','watch','watching','watched','HBO','Netflix','binge watch','binge watched','HULU',
                  'Amazon Video','season','prime video','television','slinc','CBS','philo','fuboTV','direct TV','Youtube TV','Youtube','playstation',
                  'xbox','wii','ESPN','Showtime','ABC','Starz','Fox','binge','Pluto TV', 'lazy','cozy','blanket','pillow']
print('Number of unhealthy activities tested:',len(unhealthyActivities))
unhealthy_acts_list=[]
for act in unhealthyActivities:
    unhealthyact_tweets=scroll_tweets(query=act, limit=2000)
    unhealthy_acts_list.append(unhealthyact_tweets)
uha_t_l = [item for sublist in unhealthy_acts_list for item in sublist]
len(uha_t_l)

In [ ]:
# Let's locate those tweets
unhealthyact_tweet_df=extract_information(uha_t_l)
unhealthyact_tweet_df['Cat']='unhealthy'
unhealthyact_tweet_df=determine_USState(unhealthyact_tweet_df)
unhealthyact_tweet_df.to_csv('unhealthyACT_tweet_df_NEW.csv')

# Foods
## Fruits and veggies (66)

In [ ]:
import requests
healthyFooddict={'healthyFoods2':['banana','blackberries','blueberries','cherry','coconut','cranberry','date','fig','goji'],
                'healthyFoods3':['grape','grapefruit','kiwi','lemon','lime','lyche','mango','melon','watermelon','nectarine'],
                'healthyFoods4':['orange','papaya','passionfruit','peach','pear','plum','pineapple','pomegranate','raspberry'],
                'healthyFoods5':['star fruit','strawberry','cantaloupe','artichoke','asparagus','beans','legumes','broccoli'],
                'healthyFoods6':['brussels sprouts','cabbage','cauliflower','celery','endives','fennel','kale','spinach'],
                'healthyFoods7':['lettuce','salad','mushrooms','okra','garlic','chives',
             'beetroot','beets','ginger','radish','squash','tomato']}

healthy_food_list=[]
for k,v in healthyFooddict.items():
    print(k)
    print(len(v))
    for food in v:
        try:
            healthyfood_tweets=scroll_tweets(query=food, limit=2000)
            healthy_food_list.append(healthyfood_tweets)
        except requests.Timeout as err:
             print(food, err.message)
    #flatten the tweet list   
    hf_t_l = [item for sublist in healthy_food_list for item in sublist]
    print('Length of tweet list from', k,':', len(hf_t_l))
    # Let's locate those tweets
    healthyfood_tweet_df=extract_information(hf_t_l)
    healthyfood_tweet_df['Cat']='healthy'
    healthyfood_tweet_df=determine_USState(healthyfood_tweet_df)
    filename=k+'_tweet_df_NEW.csv'
    healthyfood_tweet_df.to_csv(filename)

## Unhealthy Foods (28)

In [ ]:
UnhealthyFoodDict={'unhealthyFood1':['lemonade','coke','soda','sprite','pepsi','pizza','fries','burger'],
                    'unhealthyFood2':['cheeseburger','cheese','cream','sauce','cupcake','cake','cookie'],
                   'unhealthyFood3':['donut','chips','syrup','candy','fudge','pie','pudding','brownie'],
                   'unhealthyFood4':['all you can eat','frozen yoghurt','chicken nuggets','waffle','pancake']}
unhealthy_food_list=[]
for k,v in UnhealthyFoodDict.items():
    print(k)
    print(len(v))
    for food in v:
        print(food)
        try:
            unhealthyfood_tweets=scroll_tweets(query=food, limit=2000)
            unhealthy_food_list.append(unhealthyfood_tweets)
        except requests.Timeout as err:
             print(food, err.message)
    #flatten the tweet list   
    uhf_t_l = [item for sublist in unhealthy_food_list for item in sublist]
    print('Length of tweet list from', k,':', len(uhf_t_l))
    # Let's locate those tweets
    unhealthyfood_tweet_df=extract_information(uhf_t_l)
    unhealthyfood_tweet_df['Cat']='unhealthy'
    unhealthyfood_tweet_df=determine_USState(unhealthyfood_tweet_df)
    filename=k+'_tweet_df_NEW.csv'
    unhealthyfood_tweet_df.to_csv(filename)